# Import packages et données

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Lecture des fichiers
dicoBuch = "D:/Antoine/CNAM/3A/WS/cnam-athon-pronunciation-main/data/dictionary/1757_Buchanan-J.csv" 
cleaner = "D:/Antoine/CNAM/3A/WS/cnam-athon-pronunciation-main/data/dictionary/Buchanan_PronChar_counts.csv"
dicoBigi = "D:/Antoine/CNAM/3A/WS/cnam-athon-pronunciation-main/data/dictionary/2015_Bigi-B.dict" 
mapper = "D:/Antoine/CNAM/3A/WS/cnam-athon-pronunciation-main/data/dictionary/sampa-ipa.txt" 

# demander à pandas de lire le fichier et garder ce tableau accessible dans une variable
dfDicoBuch = pd.read_csv(dicoBuch, delimiter=";", encoding="utf8")
dfCleaner = pd.read_csv(cleaner, delimiter=";", encoding="utf8")
dfDicoBigi = pd.read_csv(dicoBigi, delimiter=" \[\] ", encoding="utf8", engine='python', names=['hwd','pronSampa'], skiprows=8) 
dfMapper = pd.read_csv(mapper, delimiter="  *", encoding="utf8", engine='python', names=['sampa','ipa'])    

# Fonctions

In [3]:
def sizePron(strPron):
    """retourne le nombre de caractère de la prononciation"""
    return len(strPron)

In [4]:
def cleanWord(word):
    """nettoie un mot en remplaçant les caractères spéciaux d'une prononciation ou d'encodage impropre
    par les 26 lettres de l'alphabet"""
    newWord = ""
    for letter in word:
        if(isinstance(dfCleaner[dfCleaner["char"]==letter]["graph"].values[0], str)):
            newWord += dfCleaner[dfCleaner["char"]==letter]["graph"].values[0]
        
    return newWord

In [5]:
def ipatizer(word):
    """Renvoie le format IPA d'un mot"""
    newWord = ""
    for letter in word:
        if dfMapper[dfMapper["sampa"]==str(letter)].empty:
            newWord += letter
        else:
            newWord += dfMapper[dfMapper["sampa"]==str(letter)]["ipa"].values[0]
    return newWord

# Application aux données

In [6]:
dfDicoBuch['sizePron'] = dfDicoBuch.apply(lambda x: sizePron(x['pron']),1)

In [7]:
dfDicoBuch['motVedette'] = dfDicoBuch.apply(lambda x: cleanWord(x['pron']),1)

In [8]:
dfDicoBuch = dfDicoBuch.merge(dfDicoBigi, how='left',left_on='motVedette', right_on='hwd')

In [9]:
dfDicoBuch.head()

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron,motVedette,hwd,pronSampa
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,7,abacus,abacus,{ b @ k @ s
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19,6,abaft,NaN,NaN
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19,10,abaisance,NaN,NaN
3,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,8,abandon,abandon,@ b { n d @ n
4,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,10,abandoned,abandoned,@ b { n d @ n d


In [10]:
dfDicoBuch["ipa"] = [ipatizer(word.replace(" ", ",").split(",")) if (type(word) == str) else "" for word in dfDicoBuch["pronSampa"]]

In [11]:
dfDicoBuch.head(10)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron,motVedette,hwd,pronSampa,ipa
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,7,abacus,abacus,{ b @ k @ s,æbəkəs
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19,6,abaft,NaN,NaN,
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19,10,abaisance,NaN,NaN,
3,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,8,abandon,abandon,@ b { n d @ n,əbændən
4,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,10,abandoned,abandoned,@ b { n d @ n d,əbændənd
5,2,ăbāsʹe,F.,NaN,NaN,v.,7.0,19,6,abase,NaN,NaN,
6,3,ăbāʹſ_e_ment,F.,NaN,NaN,n.,8.0,19,12,abasement,NaN,NaN,
7,2,ăbăʹſh,F.,NaN,NaN,v.,9.0,19,6,abash,abash,@ b { S,əbæʃ
8,2,ăbāʹte,F.,NaN,NaN,v.,10.0,19,6,abate,abate,@ b eI t,əbeIt
9,3,ăbāʹt_e_ment,F.,NaN,NaN,n.,11.0,19,12,abatement,abatement,@ b eI t m @ n t,əbeItmənt


# Export

In [12]:
pathDicoBigiRich = "D:/Antoine/CNAM/3A/WS/cnam-athon-pronunciation-main/data/dataframe/BigiRichSample.csv"
dfDicoBuch
dfDicoBuch.to_csv(pathDicoBigiRich, sep=";", encoding="utf8", index=False)